In [1]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

In [2]:
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# Update with your API key
os.environ["LANGCHAIN_API_KEY"] = "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"
# Update with your API URL if using a hosted instance of Langsmith.
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [4]:
llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model="llama3-8b-8192"
)

In [5]:
template = """
    Olá, você é um assistente útil e gentil. Ajude as pessoas no que você conseguir.
    
    Responda esta pergunta do usuário: {user_query}
"""

prompt = PromptTemplate.from_template(template)

chain = prompt | llm | StrOutputParser()

In [6]:
print(chain.invoke({"user_query": "Olá, você pode me explicar o que é Reinforcement Learning?"}))

Olá!

Sim, com prazer! Reinforcement Learning (RL) é um subcampo da Intelligença Artificial que se concentra em treinar agentes para aprender a tomar decisões com base em recompensas ou punições recebidas ao longo do tempo. O objetivo é maximizar a recompensa e minimizar a penalidade, permitindo que o agente desenvolva estratégias para resolver problemas complexos.

Nesse sentido, o agente aprende a interagir com o ambiente, explorando diferentes ações e observando as consequências delas. Com base nessa experiência, o agente ajusta suas ações para maximizar a recompensa e evitar as penalidades.

Um exemplo clássico de RL é o agente que precisa aprender a controlar um robô para coletar alimentos em um ambiente com obstáculos. O agente aprende a evitar os obstáculos e coletar os alimentos ao mesmo tempo, recebendo recompensas quando consegue isso.

RL é amplamente aplicado em áreas como:

1. Jogos: desenvolver agentes que possam jogar jogos como Go, Poker, etc.
2. Automação industrial: t

Failed to multipart ingest runs: Connection error caused failure to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. Please confirm your internet connection. ConnectionError(MaxRetryError('HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Max retries exceeded with url: /runs/multipart (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000137071D0A00>: Failed to resolve \'api.smith.langchain.com\' ([Errno 11001] getaddrinfo failed)"))'))
Content-Length: 50359
API Key: lsv2_********************************************78trace=01acf5b5-fc82-49d4-96c4-5016baf7170b,id=01acf5b5-fc82-49d4-96c4-5016baf7170b; trace=01acf5b5-fc82-49d4-96c4-5016baf7170b,id=07114f7a-a7fe-4fb4-addd-45614db48c85; trace=01acf5b5-fc82-49d4-96c4-5016baf7170b,id=5b736de5-332a-4c5c-b9b9-e80810c12c28; trace=01acf5b5-fc82-49d4-96c4-5016baf7170b,id=67decab5-8fd7-4e60-b659-68aa9e56ad97; trace=01acf5b5-fc82-49d4-96c4-5016baf7170b,id=62ec2d94-9982-4643-a379-fded

In [15]:
from langchain_openai import ChatOpenAI

In [16]:
from pydantic import BaseModel
from typing import List, Optional

class OutputTaxonomy(BaseModel):
    type: str
    value: str

In [17]:
template = """
    Você é um classificador de taxonomias de transações bancárias. 
    Dado uma transação bancária, você precisa classificar a qual tipo de taxonomia essa transação pertence.
    
    Por exemplo, se eu pedi um Uber para me locomover e os dados da transação são:
    
    "Uber *Uber *Trip | R$ 44,99"
    
    A classificação deve ser formatada como:
    
    type: 'Transporte'
    value: 'R$ 44,99'
    
    Portanto, classifique a transação a seguir:
    
    ------
    TRANSAÇÃO: {transacao}
"""

prompt = PromptTemplate.from_template(template)

In [20]:
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4o")
llm_with_tools = llm.bind_tools([OutputTaxonomy])#, strict=True)

In [21]:
chain = prompt | llm_with_tools

In [23]:
result = chain.invoke({"transacao": "Ifood | R$ 21,17"})
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_i6RO4OBSUYCPvhe2mtzs8jth', 'function': {'arguments': '{"type":"Alimentação","value":"R$ 21,17"}', 'name': 'OutputTaxonomy'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 182, 'total_tokens': 208, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_159d8341cc', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-26e52a13-0e40-44ee-8c8d-0b4ed5648a5f-0', tool_calls=[{'name': 'OutputTaxonomy', 'args': {'type': 'Alimentação', 'value': 'R$ 21,17'}, 'id': 'call_i6RO4OBSUYCPvhe2mtzs8jth', 'type': 'tool_call'}], usage_metadata={'input_tokens': 182, 'output_tokens': 26, 'total_tokens': 208, 'input_token_details': {'audio': 0, 'cache_read': 

In [33]:
extract = result.additional_kwargs['tool_calls'][0]['function']['arguments']

In [34]:
eval(extract)

{'type': 'Alimentação', 'value': 'R$ 21,17'}

In [35]:
robust_template = """
Você é um especialista em classificação de transações bancárias, treinado para categorizar precisamente qualquer tipo de transação financeira em categorias padronizadas. Sua função é analisar o texto da transação e determinar sua categoria mais apropriada, mesmo quando a descrição não for óbvia ou contiver abreviações.

## Categorias Principais:
- Alimentação (restaurantes, mercados, delivery de comida)
- Transporte (uber, taxi, combustível, passagens)
- Moradia (aluguel, condomínio, luz, água, gás)
- Saúde (farmácia, consultas médicas, plano de saúde)
- Educação (cursos, material escolar, mensalidades)
- Lazer (streaming, jogos, eventos, shopping)
- Serviços (assinaturas, serviços domésticos, internet)
- Investimentos (aplicações, poupança)
- Receitas (salário, rendimentos, reembolsos)
- Outros (transferências, saques)

## Regras de Classificação:
1. Analise o nome do estabelecimento/serviço
2. Considere o valor da transação como contexto
3. Quando houver ambiguidade, escolha a categoria mais provável
4. Para estabelecimentos não familiares, tente inferir pelo nome/contexto
5. Mantenha consistência nas classificações similares

## Exemplos de Classificação:

Input: "IFOOD *IFOOD* | R$ 35,90"
Output:
type: 'Alimentação'
value: 'R$ 35,90'

Input: "99*99*TAXI | R$ 24,50"
Output:
type: 'Transporte'
value: 'R$ 24,50'

Input: "NETFLIX.COM | R$ 39,90"
Output:
type: 'Lazer'
value: 'R$ 39,90'

Input: "DRG*DROGASIL | R$ 89,90"
Output:
type: 'Saúde'
value: 'R$ 89,90'

Input: "GYMPASS | R$ 59,90"
Output:
type: 'Saúde'
value: 'R$ 59,90'

Input: "PIX TRANSFER | R$ 1000,00"
Output:
type: 'Outros'
value: 'R$ 1000,00'

Input: "Oxxo Quantinper"
Output:
type: 'Alimentação'
value: 'R$ 4,29'

## Instruções para Processamento:
1. Leia atentamente a descrição da transação
2. Identifique palavras-chave e padrões conhecidos
3. Compare com os exemplos fornecidos
4. Selecione a categoria mais apropriada
5. Retorne SEMPRE no formato especificado:
   type: 'Categoria'
   value: 'R$ XX,XX'

Agora, classifique a seguinte transação:
{transacao}
"""

In [37]:
robust_prompt = PromptTemplate.from_template(robust_template)
llm_robust_with_tools = llm.bind_tools([OutputTaxonomy])

chain_robust = robust_prompt | llm_robust_with_tools

In [46]:
transacoes_dificeis = {
    # Transações de Alimentação com nomes ambíguos
    "GOLDEN CROSS SP": "R$ 42,90",  # Padaria/Confeitaria, pode confundir com plano de saúde
    "QUIOSQUE VERDE": "R$ 15,50",   # Sucos/Lanches, pode confundir com serviços de jardinagem
    "BOX 217": "R$ 68,30",          # Restaurant em galeria, pode parecer loja
    
    # Transações de Transporte com nomes não óbvios
    "MOBIZAP SERVICOS": "R$ 23,45",     # Serviço de patinetes elétricos
    "EASY CARONA": "R$ 34,90",          # Serviço de carona compartilhada
    "YELLOMOVE": "R$ 19,90",            # Aluguel de bicicletas
    
    # Serviços digitais com nomes crípticos
    "MOBITECH*BR": "R$ 59,90",          # Streaming de música
    "FASTPASS*": "R$ 29,90",            # Assinatura de delivery
    "CLOUDPAY*GAMES": "R$ 99,90",       # Plataforma de jogos
    
    # Estabelecimentos com múltiplas possibilidades
    "VILA VERDE": "R$ 156,70",          # Pode ser restaurante, loja de plantas ou condomínio
    "SPAZIO": "R$ 89,90",               # Academia, restaurante ou loja
    "CENTRAL BOX": "R$ 245,30",         # Pode ser crossfit, coworking ou loja
    
    # Nomes abreviados ou truncados
    "PAG*MVUP*BR": "R$ 49,90",          # Aplicativo de mobilidade urbana
    "INT*STRY*APP": "R$ 19,90",         # Aplicativo de histórias
    "WSH*LOC*SER": "R$ 78,50",          # Serviço de lavanderia
    
    # Estabelecimentos internacionais com nomes não óbvios
    "BYTEGO*INT": "R$ 45,90",           # App de edição de vídeos
    "QUICKMED*": "R$ 199,90",           # Telemedicina
    "GREENFIN*TECH": "R$ 29,90",        # App de educação financeira
    
    # Serviços com nomes que podem confundir
    "VIDA MAIS": "R$ 89,90",            # Pode ser academia, plano de saúde ou loja natural
    "SELFCLUB": "R$ 119,90",            # Pode ser academia, clube de benefícios ou loja
    "BOA FORMA PAG": "R$ 150,00",       # Academia ou revista
    
    # Estabelecimentos com variações de nome
    "IFOOD*JACUI123": "R$ 45,50",       # Restaurante no iFood com código
    "MP *BOAPRACA": "R$ 67,80",         # Mercado Pago em estabelecimento
    "PAG*ESTAC*12": "R$ 25,00",         # Estacionamento com código
    
    "Pagsmile Intermediacao de Negocios LTDA -23.010.551/0001-31 - MICROCASH SCMEPP LTDA.": "66,90",
    "Pedro Henrique Almeida Vidal - •••.873.523-•• - NU": "800,00",
    "Notorious Fish": "92,28",
    "Rest Casa Amarela": "43,15"
}

In [47]:
result = chain_robust.invoke({"transacao": transacoes_dificeis})

In [48]:
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_qoaooIjkBGPIRKCkD5Vdf0Gf', 'function': {'arguments': '{"type": "Saúde", "value": "R$ 42,90"}', 'name': 'OutputTaxonomy'}, 'type': 'function'}, {'id': 'call_J8X62THTNldQFub5KhRKFi7M', 'function': {'arguments': '{"type": "Alimentação", "value": "R$ 15,50"}', 'name': 'OutputTaxonomy'}, 'type': 'function'}, {'id': 'call_3bDKIIdCglVvkQF6rrmVgTQK', 'function': {'arguments': '{"type": "Outros", "value": "R$ 68,30"}', 'name': 'OutputTaxonomy'}, 'type': 'function'}, {'id': 'call_fHVhhIcbzmIglxhxaskicuFV', 'function': {'arguments': '{"type": "Transporte", "value": "R$ 23,45"}', 'name': 'OutputTaxonomy'}, 'type': 'function'}, {'id': 'call_jdQ6Zp5wvJjFTpfxUVvGiwyD', 'function': {'arguments': '{"type": "Transporte", "value": "R$ 34,90"}', 'name': 'OutputTaxonomy'}, 'type': 'function'}, {'id': 'call_2HNPaOEf6CCZeVdOHMcHe1Eu', 'function': {'arguments': '{"type": "Transporte", "value": "R$ 19,90"}', 'name': 'OutputTaxonomy'}, 'type':

In [49]:
extract = result.additional_kwargs['tool_calls']
extract

[{'id': 'call_qoaooIjkBGPIRKCkD5Vdf0Gf',
  'function': {'arguments': '{"type": "Saúde", "value": "R$ 42,90"}',
   'name': 'OutputTaxonomy'},
  'type': 'function'},
 {'id': 'call_J8X62THTNldQFub5KhRKFi7M',
  'function': {'arguments': '{"type": "Alimentação", "value": "R$ 15,50"}',
   'name': 'OutputTaxonomy'},
  'type': 'function'},
 {'id': 'call_3bDKIIdCglVvkQF6rrmVgTQK',
  'function': {'arguments': '{"type": "Outros", "value": "R$ 68,30"}',
   'name': 'OutputTaxonomy'},
  'type': 'function'},
 {'id': 'call_fHVhhIcbzmIglxhxaskicuFV',
  'function': {'arguments': '{"type": "Transporte", "value": "R$ 23,45"}',
   'name': 'OutputTaxonomy'},
  'type': 'function'},
 {'id': 'call_jdQ6Zp5wvJjFTpfxUVvGiwyD',
  'function': {'arguments': '{"type": "Transporte", "value": "R$ 34,90"}',
   'name': 'OutputTaxonomy'},
  'type': 'function'},
 {'id': 'call_2HNPaOEf6CCZeVdOHMcHe1Eu',
  'function': {'arguments': '{"type": "Transporte", "value": "R$ 19,90"}',
   'name': 'OutputTaxonomy'},
  'type': 'functi

In [50]:
for dictio in extract:
    print(dictio['function']['arguments'])
    print("-" * 50)

{"type": "Saúde", "value": "R$ 42,90"}
--------------------------------------------------
{"type": "Alimentação", "value": "R$ 15,50"}
--------------------------------------------------
{"type": "Outros", "value": "R$ 68,30"}
--------------------------------------------------
{"type": "Transporte", "value": "R$ 23,45"}
--------------------------------------------------
{"type": "Transporte", "value": "R$ 34,90"}
--------------------------------------------------
{"type": "Transporte", "value": "R$ 19,90"}
--------------------------------------------------
{"type": "Outros", "value": "R$ 59,90"}
--------------------------------------------------
{"type": "Transporte", "value": "R$ 29,90"}
--------------------------------------------------
{"type": "Lazer", "value": "R$ 99,90"}
--------------------------------------------------
{"type": "Alimentação", "value": "R$ 156,70"}
--------------------------------------------------
{"type": "Outros", "value": "R$ 89,90"}
-------------------------

In [52]:
i = 0
for key in transacoes_dificeis:
    print(f"Transação: {key}")
    print(f"Valor: {transacoes_dificeis[key]}")
    print(f"Classificação: {eval(extract[i]['function']['arguments'])}")
    print("-" * 50)
    i += 1

Transação: GOLDEN CROSS SP
Valor: R$ 42,90
Classificação: {'type': 'Saúde', 'value': 'R$ 42,90'}
--------------------------------------------------
Transação: QUIOSQUE VERDE
Valor: R$ 15,50
Classificação: {'type': 'Alimentação', 'value': 'R$ 15,50'}
--------------------------------------------------
Transação: BOX 217
Valor: R$ 68,30
Classificação: {'type': 'Outros', 'value': 'R$ 68,30'}
--------------------------------------------------
Transação: MOBIZAP SERVICOS
Valor: R$ 23,45
Classificação: {'type': 'Transporte', 'value': 'R$ 23,45'}
--------------------------------------------------
Transação: EASY CARONA
Valor: R$ 34,90
Classificação: {'type': 'Transporte', 'value': 'R$ 34,90'}
--------------------------------------------------
Transação: YELLOMOVE
Valor: R$ 19,90
Classificação: {'type': 'Transporte', 'value': 'R$ 19,90'}
--------------------------------------------------
Transação: MOBITECH*BR
Valor: R$ 59,90
Classificação: {'type': 'Outros', 'value': 'R$ 59,90'}
--------------

### Chains com Llama + Groq

#### Análise básica

In [7]:
template_agregador = """
Você é um analista financeiro especializado em agregação de dados. A partir das transações classificadas fornecidas, você deve:

1. Calcular o total gasto em cada categoria
2. Identificar as maiores despesas individuais
3. Calcular médias diárias/semanais/mensais de gastos
4. Identificar períodos de maior gasto
5. Destacar comportamentos positivos a reforçar.

A ideia é que você agregue e analise os dados das transações classificadas para fornecer insights financeiros úteis. Detalhe a agragação.

Dados das transações:
{transacoes}
"""

In [8]:
template_padroes = """
Como especialista em análise de comportamento financeiro, analise os dados agregados para identificar:

1. Padrões de gastos recorrentes
2. Ciclos de despesas (início/fim de mês, fins de semana)
3. Categorias com gastos crescentes
4. Relações entre diferentes tipos de despesas

Detalhe os padrões e comportamentos identificados. A ideia é que esta seja uma análise básica da situação financeira do usuário.

Dados agregados:
{dados_agregados}
"""

#### Análise avançada

In [9]:
template_tendencias = """
Como especialista em análise de tendências financeiras, examine os padrões identificados para:

1. Projetar gastos futuros por categoria
2. Identificar tendências de aumento/diminuição de gastos
3. Comparar com benchmarks de gastos saudáveis
4. Avaliar sustentabilidade dos padrões atuais

Detalhe bem as tendências identificadas e as projeções futuras. A ideia é que tenhamos o máximo de conhecimento possível do indíviduo para ajudá-lo a tomar decisões financeiras mais conscientes.

Dados de padrões:
{padroes}
"""

In [10]:
template_anomalias = """
Como especialista em detecção de anomalias financeiras, analise os dados e os padrões do usuário para identificar:

1. Gastos fora do padrão habitual
2. Possíveis cobranças duplicadas
3. Serviços recorrentes subutilizados
4. Categorias com gastos desproporcionais

Detalhe bem as anomalias identificadas e sugira ações corretivas. A ideia é que possamos ajudar o usuário a identificar e corrigir problemas financeiros antes que se tornem graves. Se não houver 
anomalias, informe que não foram encontradas.

Dados:
{dados_completos}
Padrões:
{padroes}
"""

#### Geração de insights

In [11]:
template_insights = """
Como consultor financeiro, analise as tendências de comportamento do usuário, seus padrões e anomalias possivelmente identificadas para gerar insights acionáveis:

1. Principais áreas de oportunidade de economia
2. Comportamentos financeiros positivos a manter
3. Comportamentos que precisam de atenção
4. Comparação com metas financeiras (se estabelecidas)

Detalhe bem os insights gerados e sugira ações específicas para melhorar a saúde financeira do usuário. A ideia é que esses insights possam ser analisados por um consultor a fim do mesmo
criar recomendações personalizadas baseadas nestes insights.

Tendências:
{dados_completos}
Anomalias:
{anomalias}
"""

In [12]:
template_recomendacoes = """
Como consultor financeiro pessoal, crie recomendações personalizadas baseadas nos insights:

1. Sugestões específicas de economia por categoria
2. Alternativas para serviços caros identificados
3. Estratégias para melhorar hábitos financeiros
4. Metas sugeridas para o próximo período

Detalhes bem as recomendações e sugira ações específicas para melhorar a saúde financeira do usuário. A ideia é que essas recomendações sejam práticas e fáceis de seguir, ajudando o usuário a 
melhorar sua situação financeira.

Insights disponíveis:
{insights}
"""

#### Composição do relatório

In [13]:
template_relatorio = """
Como especialista em comunicação financeira, crie um relatório completo e amigável que inclua:

1. Resumo Executivo
   - Principais números e conclusões
   - Destaques do período

2. Análise Detalhada
   - Breakdown por categorias
   - Tendências identificadas
   - Anomalias importantes

3. Recomendações
   - Sugestões práticas
   - Próximos passos sugeridos

Dados completos:
{dados_completos}
Insights:
{insights}
Recomendações:
{recomendacoes}

Crie um relatório em formato Markdown que seja informativo mas não intimidante.
"""

In [14]:
prompt_agregador = PromptTemplate.from_template(template_agregador)
prompt_padroes = PromptTemplate.from_template(template_padroes)
prompt_tendencias = PromptTemplate.from_template(template_tendencias)
prompt_anomalias = PromptTemplate.from_template(template_anomalias)
prompt_insights = PromptTemplate.from_template(template_insights)
prompt_recomendacoes = PromptTemplate.from_template(template_recomendacoes)
prompt_relatorio = PromptTemplate.from_template(template_relatorio)

In [15]:
transacoes_teste = """
Transação: GOLDEN CROSS SP
Valor: R$ 42,90
Classificação: {'type': 'Saúde', 'value': 'R$ 42,90'}
--------------------------------------------------
Transação: QUIOSQUE VERDE
Valor: R$ 15,50
Classificação: {'type': 'Alimentação', 'value': 'R$ 15,50'}
--------------------------------------------------
Transação: BOX 217
Valor: R$ 68,30
Classificação: {'type': 'Outros', 'value': 'R$ 68,30'}
--------------------------------------------------
Transação: MOBIZAP SERVICOS
Valor: R$ 23,45
Classificação: {'type': 'Transporte', 'value': 'R$ 23,45'}
--------------------------------------------------
Transação: EASY CARONA
Valor: R$ 34,90
Classificação: {'type': 'Transporte', 'value': 'R$ 34,90'}
--------------------------------------------------
Transação: YELLOMOVE
Valor: R$ 19,90
Classificação: {'type': 'Transporte', 'value': 'R$ 19,90'}
--------------------------------------------------
Transação: MOBITECH*BR
Valor: R$ 59,90
Classificação: {'type': 'Outros', 'value': 'R$ 59,90'}
--------------------------------------------------
Transação: FASTPASS*
Valor: R$ 29,90
Classificação: {'type': 'Transporte', 'value': 'R$ 29,90'}
--------------------------------------------------
Transação: CLOUDPAY*GAMES
Valor: R$ 99,90
Classificação: {'type': 'Lazer', 'value': 'R$ 99,90'}
--------------------------------------------------
Transação: VILA VERDE
Valor: R$ 156,70
Classificação: {'type': 'Alimentação', 'value': 'R$ 156,70'}
--------------------------------------------------
Transação: SPAZIO
Valor: R$ 89,90
Classificação: {'type': 'Outros', 'value': 'R$ 89,90'}
--------------------------------------------------
Transação: CENTRAL BOX
Valor: R$ 245,30
Classificação: {'type': 'Outros', 'value': 'R$ 245,30'}
--------------------------------------------------
Transação: PAG*MVUP*BR
Valor: R$ 49,90
Classificação: {'type': 'Outros', 'value': 'R$ 49,90'}
--------------------------------------------------
Transação: INT*STRY*APP
Valor: R$ 19,90
Classificação: {'type': 'Serviços', 'value': 'R$ 19,90'}
--------------------------------------------------
Transação: WSH*LOC*SER
Valor: R$ 78,50
Classificação: {'type': 'Outros', 'value': 'R$ 78,50'}
--------------------------------------------------
Transação: BYTEGO*INT
Valor: R$ 45,90
Classificação: {'type': 'Serviços', 'value': 'R$ 45,90'}
--------------------------------------------------
Transação: QUICKMED*
Valor: R$ 199,90
Classificação: {'type': 'Saúde', 'value': 'R$ 199,90'}
--------------------------------------------------
Transação: GREENFIN*TECH
Valor: R$ 29,90
Classificação: {'type': 'Outros', 'value': 'R$ 29,90'}
--------------------------------------------------
Transação: VIDA MAIS
Valor: R$ 89,90
Classificação: {'type': 'Saúde', 'value': 'R$ 89,90'}
--------------------------------------------------
Transação: SELFCLUB
Valor: R$ 119,90
Classificação: {'type': 'Saúde', 'value': 'R$ 119,90'}
--------------------------------------------------
Transação: BOA FORMA PAG
Valor: R$ 150,00
Classificação: {'type': 'Saúde', 'value': 'R$ 150,00'}
--------------------------------------------------
Transação: IFOOD*JACUI123
Valor: R$ 45,50
Classificação: {'type': 'Alimentação', 'value': 'R$ 45,50'}
--------------------------------------------------
Transação: MP *BOAPRACA
Valor: R$ 67,80
Classificação: {'type': 'Alimentação', 'value': 'R$ 67,80'}
--------------------------------------------------
Transação: PAG*ESTAC*12
Valor: R$ 25,00
Classificação: {'type': 'Serviços', 'value': 'R$ 25,00'}
--------------------------------------------------
Transação: Pagsmile Intermediacao de Negocios LTDA -23.010.551/0001-31 - MICROCASH SCMEPP LTDA.
Valor: 66,90
Classificação: {'type': 'Outros', 'value': '66,90'}
--------------------------------------------------
Transação: Pedro Henrique Almeida Vidal - •••.873.523-•• - NU
Valor: 800,00
Classificação: {'type': 'Outros', 'value': '800,00'}
--------------------------------------------------
Transação: Notorious Fish
Valor: 92,28
Classificação: {'type': 'Alimentação', 'value': '92,28'}
--------------------------------------------------
Transação: Rest Casa Amarela
Valor: 43,15
Classificação: {'type': 'Alimentação', 'value': '43,15'}
--------------------------------------------------
"""

In [ ]:
llm = ChatGroq(
    api_key=GROQ_API_KEY,
    model="llama3-8b-8192"
)

chain_agregador = prompt_agregador | llm | StrOutputParser()
chain_padroes = prompt_padroes | llm | StrOutputParser()
chain_tendencias = prompt_tendencias | llm | StrOutputParser()
chain_anomalias = prompt_anomalias | llm | StrOutputParser()
chain_insights = prompt_insights | llm | StrOutputParser()
chain_recomendacoes = prompt_recomendacoes | llm | StrOutputParser()
chain_relatorio = prompt_relatorio | llm | StrOutputParser()

chain_analise_basica = {"dados_agregados": chain_agregador} | chain_padroes
# print(chain_analise_basica.invoke({"transacoes": transacoes_teste}))

chain_analise_tendencia = {"padroes": chain_analise_basica} | chain_tendencias
# print(chain_analise_tendencia.invoke({"transacoes": transacoes_teste}))

chain_analise_anomalias = {"padroes": chain_analise_basica} | RunnablePassthrough() | {"dados_completos": chain_tendencias, "padroes": itemgetter("padroes")} | chain_anomalias
# print(chain_analise_anomalias.invoke({"transacoes": transacoes_teste}))

chain_gera_insights = (
    {"padroes": chain_analise_basica} 
    | RunnablePassthrough() 
    | {"dados_completos": chain_tendencias, "padroes": itemgetter("padroes")} 
    | {"anomalias": chain_anomalias, "dados_completos": itemgetter("dados_completos")}
    | chain_insights
)
# print(chain_gera_insights.invoke({"transacoes": transacoes_teste}))

chain_gera_recomendacoes = (
    {"padroes": chain_analise_basica} 
    | RunnablePassthrough() 
    | {"dados_completos": chain_tendencias, "padroes": itemgetter("padroes")} 
    | {"anomalias": chain_anomalias, "dados_completos": itemgetter("dados_completos")}
    | {"insights": chain_insights}
    | chain_recomendacoes
)
# print(chain_gera_recomendacoes.invoke({"transacoes": transacoes_teste}))

chain_gera_relatorio = (
    {"padroes": chain_analise_basica} 
    | RunnablePassthrough() 
    | {"dados_completos": chain_tendencias, "padroes": itemgetter("padroes")} 
    | {"anomalias": chain_anomalias, "dados_completos": itemgetter("dados_completos")}
    | RunnablePassthrough() 
    | {"insights": chain_insights, "dados_completos": itemgetter("dados_completos")}
    | {"recomendacoes": chain_recomendacoes, "insights": itemgetter("insights"), "dados_completos": itemgetter("dados_completos")}
    | chain_relatorio
)
print(chain_gera_relatorio.invoke({"transacoes": transacoes_teste}))

**Relatório de Análise de Tendências Financeiras**

**Resumo Executivo**
-------------------

O relatório apresenta uma análise detalhada dos gastos do usuário, identificando tendências, anomalias e recomendações para melhorar a saúde financeira. Os principais números e conclusões são apresentados abaixo:

* Gastos totais: R$ 5.000,00
* Projeção de gastos futuros: R$ 5.300,00
* Principais categorias de gastos: Alimentação (30%), Outros (20%), Saúde (15%)

**Análise Detalhada**
--------------------

### Breakdown por categorias

* Alimentação: R$ 1.500,00 (30% dos gastos totais)
* Outros: R$ 1.000,00 (20% dos gastos totais)
* Saúde: R$ 750,00 (15% dos gastos totais)

### Tendências identificadas

* Tendência a aumentar os gastos em Alimentação e Outros
* Tendência a diminuir os gastos em Saúde e Transporte

### Anomalias importantes

* Gastos excessivos em Alimentação e Outros
* Gastos baixos em Saúde

**Recomendações**
----------------

### Sugestões práticas

* Reduzir gastos excessiv

In [25]:
chain_gera_relatorio = (
    {"padroes": chain_analise_basica} 
    | RunnablePassthrough() 
    | {"dados_completos": chain_tendencias, "padroes": itemgetter("padroes")} 
    | {"anomalias": chain_anomalias, "dados_completos": itemgetter("dados_completos")}
    | {"insights": chain_insights, "dados_completos": itemgetter("dados_completos")}
    | {"recomendacoes": chain_recomendacoes, "insights": itemgetter("insights"), "dados_completos": itemgetter("dados_completos")}
    | chain_relatorio
)
print(chain_gera_relatorio.invoke({"transacoes": transacoes_teste}))

**Relatório de Análise de Gastos**

**Resumo Executivo**

O presente relatório apresenta uma análise detalhada dos padrões de gastos do usuário, identificando tendências, oportunidades de economia e recomendações para melhoria. Os principais números e conclusões são apresentados abaixo:

* O gasto com alimentação é projetado para aumentar em 9,5% no próximo período.
* O gasto com saúde é projetado para aumentar em 11,1% no próximo período.
* O gasto com transporte é projetado para aumentar em 5% no próximo período.
* A sustentabilidade dos padrões atuais parece boa, considerando a priorização da saúde e a alimentação, bem como a gestão financeira eficiente em outras categorias.

**Análise Detalhada**

A seguir, é apresentada a análise detalhada dos padrões de gastos do usuário, incluindo breakdown por categorias, tendências identificadas e anomalias importantes.

### Breakdown por categorias

* Alimentação: projetado R$ 400,00 (15 transações)
* Saúde: projetado R$ 700,00 (6 transações)